## D7041E - Mini project: Human Real-time Detection

### Group: Laura Bermejo, Lina Borg, Julie Labbé

#### References and sources
For the training of our model, we got inspired by this video: https://www.youtube.com/watch?v=V4Kkrz__hvo.

#### Requirements
The required libraries are super-gradient (for model), roboflow (for dataset) and supervision (for bounding boxes).

pip install -q super-gradients <br>
pip install -q roboflow <br>
pip install -q supervision

In [1]:
#%pip install -q super-gradients
#%pip install -q roboflow
#%pip install -q supervision

#### Dataset

##### Code to download the dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="DNm47rTel6WBxKZqAd9T")
project = rf.workspace("capricon").project("human-detection-q0nit")
dataset = project.version(2).download("yolov5")

#### class Config Class definition

In [3]:
from typing import List, Dict

class config:
  #Project paths
  DATA_DIR: str = "Human-Detection-2"
  CHECKPOINT_DIR: str = "checkpoints"
  EXPERIMENT_NAME: str = "Project"

  #Datasets
  TRAIN_IMAGES_DIR: str ="train/images"
  TRAIN_LABELS_DIR: str = "train/labels"
  VAL_IMAGES_DIR: str = "valid/images"
  VAL_LABELS_DIR: str = "valid/labels"
  TEST_IMAGES_DIR: str = "test/images"
  TEST_LABELS_DIR: str = "test/labels"

  #Classes
  CLASSES: List[str] = ["human", "vehicle"]
  NUM_CLASSES: int = 2

  #Model 1
  # DEFINE HYPERPARAMETERS, YOU WILL HAVE TO CHANGE IT
  DATALOADER_PARAMS: Dict = {
      'batch_size':16,
      'num_workers':0
  }
  # THIS IS ALREADY SET, CHANGE ONLY DATALOADER PARAMS
  MODEL_NAME: str = 'yolo_nas_l'
  PRETRAINED_WEIGHTS: str ='coco'

#### Dataloaders initialization

In [ ]:
from IPython.display import clear_output
from typing import List, Dict
from super_gradients.training import models
from super_gradients.training import Trainer
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_val
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from typing import List, Dict
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_val

In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

#### Visualization

In [ ]:
val_data.dataset.plot()

#### Training hyperparameters

In [7]:
# YOU HAVE TO SET THE HYPERPARAMETERS IN THIS VARIABLE
train_params = {
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 10,
    "mixed_precision": True,
    "loss" : PPYoloELoss (
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=config.NUM_CLASSES,
        reg_max=16
    ),

    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=config.NUM_CLASSES,
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.8
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

#### Training

##### Download the model

In [8]:
model = models.get(config.MODEL_NAME, num_classes=config.NUM_CLASSES, pretrained_weights=config.PRETRAINED_WEIGHTS)

[2024-01-08 18:47:27] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-01-08 18:47:29] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


##### Initialize training

In [9]:
trainer = Trainer(experiment_name=config.EXPERIMENT_NAME, ckpt_root_dir=config.CHECKPOINT_DIR)

##### Train

In [10]:
trainer.train(model=model, training_params=train_params, train_loader=train_data, valid_loader=val_data)

[2024-01-08 18:47:29] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20240108_184729_542040`
[2024-01-08 18:47:29] INFO - sg_trainer.py - Checkpoints directory: checkpoints\Project\RUN_20240108_184729_542040


The console stream is now moved to checkpoints\Project\RUN_20240108_184729_542040/console_Jan08_18_47_29.txt


[2024-01-08 18:47:30] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
initialize_param_groups and update_param_groups usages are deprecated since 3.4.0, will be removed in 3.5.0 and have no effect. 
 Assign different learning rates by passing a mapping of layer name prefixes to lr values through initial_lr training hyperparameter (i.e initial_lr={'backbone': 0.01, 'default':0.1})
Mixed precision training is not supported on CPU. Disabling mixed precision. (i.e. `mixed_precision=False`)
[2024-01-08 18:47:36] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               0          (0 available on the machine)
    - Full dataset size:            159        (len(train_set))
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.3789
│   ├── Ppyoloeloss/loss_iou = 0.9081
│   ├── Ppyoloeloss/loss_dfl = 0.837
│   └── Ppyoloeloss/loss = 4.1239
└── Validation
    ├── Ppyoloeloss/loss_cls = 2.8025
    ├── Ppyoloeloss/loss_iou = 0.6772
    ├── Ppyoloeloss/loss_dfl = 0.7199
    ├── Ppyoloeloss/loss = 4.1997
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.0
    └── F1@0.50 = 0.0



Validating epoch 1: 100%|██████████| 9/9 [03:26<00:00, 22.92s/it]
[2024-01-08 20:04:08] INFO - base_sg_logger.py - Checkpoint saved in checkpoints\Project\RUN_20240108_184729_542040\ckpt_best.pth
[2024-01-08 20:04:08] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.4026811122894287


...

Validating epoch 8: 100%|██████████| 9/9 [03:02<00:00, 20.28s/it]
[2024-01-08 23:10:14] INFO - base_sg_logger.py - Checkpoint saved in checkpoints\Project\RUN_20240108_184729_542040\ckpt_best.pth
[2024-01-08 23:10:14] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.7183704376220703


SUMMARY OF EPOCH 8
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7756
│   │   ├── Epoch N-1      = 0.784  (↘ -0.0084)
│   │   └── Best until now = 0.784  (↘ -0.0084)
│   ├── Ppyoloeloss/loss_iou = 0.4715
│   │   ├── Epoch N-1      = 0.4968 (↘ -0.0253)
│   │   └── Best until now = 0.4968 (↘ -0.0253)
│   ├── Ppyoloeloss/loss_dfl = 0.455
│   │   ├── Epoch N-1      = 0.4643 (↘ -0.0093)
│   │   └── Best until now = 0.4643 (↘ -0.0093)
│   └── Ppyoloeloss/loss = 1.7021
│       ├── Epoch N-1      = 1.7452 (↘ -0.043)
│       └── Best until now = 1.7452 (↘ -0.043)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.8502
    │   ├── Epoch N-1      = 2.6827 (↘ -0.8325)
    │   └── Best until now = 2.5469 (↘ -0.6967)
    ├── Ppyoloeloss/loss_iou = 0.4174
    │   ├── Epoch N-1      = 0.4268 (↘ -0.0094)
    │   └── Best until now = 0.4268 (↘ -0.0094)
    ├── Ppyoloeloss/loss_dfl = 0.4293
    │   ├── Epoch N-1      = 0.4426 (↘ -0.0133)
    │   └── Best until now = 0.4426 (↘ -0.0133)
    ├── Ppyoloeloss/loss 

Validating epoch 9: 100%|██████████| 9/9 [03:01<00:00, 20.17s/it]
[2024-01-08 23:33:17] INFO - base_sg_logger.py - Checkpoint saved in checkpoints\Project\RUN_20240108_184729_542040\ckpt_best.pth
[2024-01-08 23:33:17] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.7298129796981812


SUMMARY OF EPOCH 9
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7621
│   │   ├── Epoch N-1      = 0.7756 (↘ -0.0135)
│   │   └── Best until now = 0.7756 (↘ -0.0135)
│   ├── Ppyoloeloss/loss_iou = 0.4783
│   │   ├── Epoch N-1      = 0.4715 (↗ 0.0068)
│   │   └── Best until now = 0.4715 (↗ 0.0068)
│   ├── Ppyoloeloss/loss_dfl = 0.4538
│   │   ├── Epoch N-1      = 0.455  (↘ -0.0012)
│   │   └── Best until now = 0.455  (↘ -0.0012)
│   └── Ppyoloeloss/loss = 1.6942
│       ├── Epoch N-1      = 1.7021 (↘ -0.0079)
│       └── Best until now = 1.7021 (↘ -0.0079)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.5286
    │   ├── Epoch N-1      = 1.8502 (↘ -0.3216)
    │   └── Best until now = 1.8502 (↘ -0.3216)
    ├── Ppyoloeloss/loss_iou = 0.4045
    │   ├── Epoch N-1      = 0.4174 (↘ -0.0129)
    │   └── Best until now = 0.4174 (↘ -0.0129)
    ├── Ppyoloeloss/loss_dfl = 0.4194
    │   ├── Epoch N-1      = 0.4293 (↘ -0.0099)
    │   └── Best until now = 0.4293 (↘ -0.0099)
    ├── Ppyoloeloss/loss

[2024-01-08 23:33:40] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validating epoch 10: 100%|██████████| 9/9 [03:00<00:00, 20.07s/it]


#### Charge best model

In [11]:
import os

avg_model = models.get(config.MODEL_NAME, num_classes=config.NUM_CLASSES, checkpoint_path=os.path.join(config.CHECKPOINT_DIR, config.EXPERIMENT_NAME, 'average_model.pth'))

[2024-01-08 23:36:42] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process


FileNotFoundError: Incorrect Checkpoint path: checkpoints\Project\average_model.pth (This should be an absolute path)

#### Test with test_dataloaders

In [ ]:
trainer.test(model=avg_model, test_loader=test_data, test_metrics_list=DetectionMetrics_050(score_thres=0.1, top_k_predictions=300, num_cls=config.NUM_CLASSES, normalize_targets=True, post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01, nms_top_k=1000, max_predictions=300)))

#### Visualization

In [ ]:
import random
import supervision as sv

random.seed(10) # THE SEED CAN BE CHANGED

##### Build dataset object

In [ ]:
ds = sv.Dataset.from_yolo(images_directory_path=f"{config.DATA_DIR}/test/images", annotations_directory_path=f"{config.DATA_DIR}/test/labels", data_yaml_path=f"{config.DATA_DIR}/data.yaml", force_masks=False)
predictions = {}
CONFIDENCE_THRESHOLD = 0.5

##### Inference

In [ ]:
for image_name,image in ds.images.items():
    result = list(avg_model.predict(image, conf=CONFIDENCE_THRESHOLD))[0]
    detections = sv.Detections(xyxy=result.prediction.bboxes_xyxy, confidence=result.prediction.confidence, class_id=result.prediction.labels.astype(int))
    predictions[image_name] = detections

##### Plot annotations vs predictions

In [ ]:
MAX_IMAGE_COUNT = 10
n = min(MAX_IMAGE_COUNT, len(ds.images))

keys = list(ds.images.keys())
keys = random.sample(keys,n)

box_annotator = sv.BoxAnnotator()

images = []
titles = []

for key in keys:
    frame_with_annotations = box_annotator.annotate(scene=ds.images[key].copy (), detections =  ds. annotations [key], skip_label=True)
    images.append(frame_with_annotations)
    titles.append('annotations')
    frame_with_predictions = box_annotator. annotate( scene = ds.images[key].copy (),detections = predictions [key], skip_label=True)
    images.append(frame_with_predictions)
    titles.append('predictions')

%matplotlib inline
sv.plot_images_grid(images-images, titles-titles, grid_size=(n, 2), size=(2*4, n*4))

#### Cross-validation
When we first downloaded the data, it got separated into three folders: test, train, validate. For cross-validation, we will disregard the generated test and validate folders. Instead, we will use `KFold` method from `scikit-learn` ...

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import KFold

def read_data(path):
    dataset = []
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        if "images" in path:
            file = cv2.imread(file_path)
        if "labels" in path:
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    file = []
                    for line in lines:
                        values = line.strip().split()
                        class_label = int(values[0])
                        bbox_coords = [float(val) for val in values[1:]]
                        annotation = [class_label] + bbox_coords
                        file.append(annotation)
                    file = np.array(file)
        dataset.append(file)
    return np.array(dataset)

def write_data(dataset, path):
    if not os.path.exists(path):
        os.makedirs(path)

    for i, file in enumerate(dataset):
        if "images" in path:
            filename = f"{i}.jpg"
            output_path = os.path.join(path, filename)
            cv2.imwrite(output_path, file)
        if "labels" in path:
            for i, annotation in enumerate(dataset):
                filename = f"{i}.txt"
                output_path = os.path.join(path, filename)

                with open(output_path, 'w') as file:
                    for value in annotation:
                        line = ' '.join(map(str, value))
                        file.write(line)
                        file.write('\n') 

def cross_validation(values, labels):
    current_iteration = 0
    best_accuracy = 0.0
    num_folds = 3
    kf = KFold(n_splits=num_folds) # TODO: change the seed here

    for train_index, test_index in kf.split(values):
        values_train, values_test = values[train_index], values[test_index]
        labels_train, labels_test = labels[train_index], labels[test_index]

        # In order to use the training function, we need to store the images and labels in respective folders
        output_path = dataset.location + f"/train_fold_{current_iteration}"
        write_data(values_train, output_path + "/images")
        write_data(labels_train, output_path + "/labels")
        current_iteration += 1
        
        # Converts our new separated datasets into Dataloaders
        train_data = coco_detection_yolo_format_train(
            dataset_params={
                'data_dir': config.DATA_DIR,
                'images_dir': "train_fold_{current_iteration}/images", 
                'labels_dir': "train_fold_{current_iteration}/labels",
                'classes': config.CLASSES
            },
            dataloader_params=config.DATALOADER_PARAMS
        )
        #predicted_labels = predict(values_test, values_train, labels_train, k)
        #accuracy = accuracy_score(labels_test, predicted_labels)
        #total_accuracy += accuracy

In [ ]:
# Load data in order to split it in folds
path = dataset.location + "/train"
loaded_dataset = read_data(path + "/images")
loaded_labels = read_data(path + "/labels")
cross_validation(loaded_dataset, loaded_labels)